In [ ]:
# Import libraries
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from src.models.ml_models import MLModels
from src.models.dl_models import DLModels
from src.evaluation.metrics import ModelEvaluator
from src.evaluation.visualization import Visualizer
from src.utils.helpers import load_config

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline
%load_ext tensorboard

## Load Preprocessed Data

In [ ]:
# Load preprocessed data
X_train = np.load('../data/processed/X_train.npy')
X_test = np.load('../data/processed/X_test.npy')
y_train = np.load('../data/processed/y_train.npy')
y_test = np.load('../data/processed/y_test.npy')

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Number of classes: {len(np.unique(y_train))}")

## 🌲 1. Train Machine Learning Models

In [ ]:
# Initialize ML models
ml_models = MLModels()

# Dictionary to store results
ml_results = {}

### Random Forest

In [ ]:
print("Training Random Forest...")

# Build model
rf_model = ml_models.build_random_forest(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2
)

# Train with cross-validation
rf_trained, rf_cv_scores = ml_models.train_model(
    rf_model,
    X_train,
    y_train,
    use_cv=True,
    cv_folds=5
)

# Predict
rf_pred, rf_pred_proba = ml_models.predict(rf_trained, X_test)

print(f"CV Accuracy: {np.mean(rf_cv_scores):.4f} (+/- {np.std(rf_cv_scores):.4f})")
print(f"Test Accuracy: {(rf_pred == y_test).mean():.4f}")

ml_results['Random Forest'] = {
    'model': rf_trained,
    'predictions': rf_pred,
    'probabilities': rf_pred_proba,
    'cv_scores': rf_cv_scores
}

### Support Vector Machine

In [ ]:
print("Training SVM...")

svm_model = ml_models.build_svm(
    kernel='rbf',
    C=1.0,
    gamma='scale'
)

svm_trained, svm_cv_scores = ml_models.train_model(
    svm_model,
    X_train,
    y_train,
    use_cv=True,
    cv_folds=5
)

svm_pred, svm_pred_proba = ml_models.predict(svm_trained, X_test)

print(f"CV Accuracy: {np.mean(svm_cv_scores):.4f} (+/- {np.std(svm_cv_scores):.4f})")
print(f"Test Accuracy: {(svm_pred == y_test).mean():.4f}")

ml_results['SVM'] = {
    'model': svm_trained,
    'predictions': svm_pred,
    'probabilities': svm_pred_proba,
    'cv_scores': svm_cv_scores
}

### XGBoost

In [ ]:
print("Training XGBoost...")

xgb_model = ml_models.build_xgboost(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8
)

xgb_trained, xgb_cv_scores = ml_models.train_model(
    xgb_model,
    X_train,
    y_train,
    use_cv=True,
    cv_folds=5
)

xgb_pred, xgb_pred_proba = ml_models.predict(xgb_trained, X_test)

print(f"CV Accuracy: {np.mean(xgb_cv_scores):.4f} (+/- {np.std(xgb_cv_scores):.4f})")
print(f"Test Accuracy: {(xgb_pred == y_test).mean():.4f}")

ml_results['XGBoost'] = {
    'model': xgb_trained,
    'predictions': xgb_pred,
    'probabilities': xgb_pred_proba,
    'cv_scores': xgb_cv_scores
}

### Gradient Boosting

In [ ]:
print("Training Gradient Boosting...")

gb_model = ml_models.build_gradient_boosting(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8
)

gb_trained, gb_cv_scores = ml_models.train_model(
    gb_model,
    X_train,
    y_train,
    use_cv=True,
    cv_folds=5
)

gb_pred, gb_pred_proba = ml_models.predict(gb_trained, X_test)

print(f"CV Accuracy: {np.mean(gb_cv_scores):.4f} (+/- {np.std(gb_cv_scores):.4f})")
print(f"Test Accuracy: {(gb_pred == y_test).mean():.4f}")

ml_results['Gradient Boosting'] = {
    'model': gb_trained,
    'predictions': gb_pred,
    'probabilities': gb_pred_proba,
    'cv_scores': gb_cv_scores
}

## 🧠 2. Train Deep Learning Models

In [ ]:
# Initialize DL models
dl_models = DLModels()

# Prepare data for DL (reshape for CNN/LSTM)
input_shape = (X_train.shape[1], 1)
num_classes = len(np.unique(y_train))

# Create validation split
val_split = 0.2
val_size = int(len(X_train) * val_split)
X_train_dl = X_train[:-val_size]
y_train_dl = y_train[:-val_size]
X_val = X_train[-val_size:]
y_val = y_train[-val_size:]

print(f"DL Training set: {X_train_dl.shape}")
print(f"DL Validation set: {X_val.shape}")

# Dictionary to store DL results
dl_results = {}

### CNN Model

In [ ]:
print("Building and training CNN...")

# Build CNN
cnn_model = dl_models.build_cnn(
    input_shape=input_shape,
    num_classes=num_classes,
    filters=[64, 128, 256],
    kernel_size=3,
    dropout_rate=0.3
)

# Compile
cnn_model = dl_models.compile_model(cnn_model, learning_rate=0.001)

# Create callbacks
cnn_callbacks = dl_models.create_callbacks(
    model_name='CNN',
    patience=10,
    save_best_only=True
)

# Train
cnn_history = dl_models.train_model(
    cnn_model,
    X_train_dl,
    y_train_dl,
    X_val,
    y_val,
    epochs=50,
    batch_size=32,
    callbacks=cnn_callbacks
)

dl_results['CNN'] = {
    'model': cnn_model,
    'history': cnn_history
}

### LSTM Model

In [ ]:
print("Building and training LSTM...")

lstm_model = dl_models.build_lstm(
    input_shape=input_shape,
    num_classes=num_classes,
    lstm_units=[128, 64],
    dropout_rate=0.3,
    bidirectional=True
)

lstm_model = dl_models.compile_model(lstm_model, learning_rate=0.001)

lstm_callbacks = dl_models.create_callbacks(
    model_name='LSTM',
    patience=10,
    save_best_only=True
)

lstm_history = dl_models.train_model(
    lstm_model,
    X_train_dl,
    y_train_dl,
    X_val,
    y_val,
    epochs=50,
    batch_size=32,
    callbacks=lstm_callbacks
)

dl_results['LSTM'] = {
    'model': lstm_model,
    'history': lstm_history
}

### VGG Model

In [ ]:
print("Building and training VGG...")

vgg_model = dl_models.build_vgg(
    input_shape=input_shape,
    num_classes=num_classes,
    num_blocks=3,
    filters_per_block=[64, 128, 256],
    dropout_rate=0.3
)

vgg_model = dl_models.compile_model(vgg_model, learning_rate=0.001)

vgg_callbacks = dl_models.create_callbacks(
    model_name='VGG',
    patience=10,
    save_best_only=True
)

vgg_history = dl_models.train_model(
    vgg_model,
    X_train_dl,
    y_train_dl,
    X_val,
    y_val,
    epochs=50,
    batch_size=32,
    callbacks=vgg_callbacks
)

dl_results['VGG'] = {
    'model': vgg_model,
    'history': vgg_history
}

### ResNet Model

In [ ]:
print("Building and training ResNet...")

resnet_model = dl_models.build_resnet(
    input_shape=input_shape,
    num_classes=num_classes,
    num_blocks=3,
    filters_per_block=[64, 128, 256],
    dropout_rate=0.3
)

resnet_model = dl_models.compile_model(resnet_model, learning_rate=0.001)

resnet_callbacks = dl_models.create_callbacks(
    model_name='ResNet',
    patience=10,
    save_best_only=True
)

resnet_history = dl_models.train_model(
    resnet_model,
    X_train_dl,
    y_train_dl,
    X_val,
    y_val,
    epochs=50,
    batch_size=32,
    callbacks=resnet_callbacks
)

dl_results['ResNet'] = {
    'model': resnet_model,
    'history': resnet_history
}

## 📊 Visualize Training History

In [ ]:
# Plot training history for all DL models
visualizer = Visualizer()

for model_name, result in dl_results.items():
    print(f"\n{model_name} Training History:")
    visualizer.plot_training_history(
        result['history'],
        title=f'{model_name} Training History'
    )
    plt.show()

## 🎯 Quick Evaluation

In [ ]:
# Evaluate ML models
evaluator = ModelEvaluator()

print("Machine Learning Models Performance:\n")
ml_comparison = []

for model_name, result in ml_results.items():
    metrics = evaluator.calculate_metrics(
        y_test,
        result['predictions'],
        result['probabilities']
    )
    ml_comparison.append({
        'Model': model_name,
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1-Score': metrics['f1_score']
    })

ml_comparison_df = pd.DataFrame(ml_comparison)
display(ml_comparison_df)

In [ ]:
# Evaluate DL models
print("\nDeep Learning Models Performance:\n")
dl_comparison = []

for model_name, result in dl_results.items():
    # Get predictions
    y_pred_proba = result['model'].predict(X_test.reshape(-1, X_test.shape[1], 1))
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    metrics = evaluator.calculate_metrics(y_test, y_pred, y_pred_proba)
    dl_comparison.append({
        'Model': model_name,
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1-Score': metrics['f1_score']
    })

dl_comparison_df = pd.DataFrame(dl_comparison)
display(dl_comparison_df)

## 💾 Save Models

In [ ]:
import os
import joblib

# Create directories
os.makedirs('../results/models/ml', exist_ok=True)
os.makedirs('../results/models/dl', exist_ok=True)

# Save ML models
for model_name, result in ml_results.items():
    filename = f"../results/models/ml/{model_name.replace(' ', '_').lower()}.pkl"
    joblib.dump(result['model'], filename)
    print(f"Saved {model_name} to {filename}")

# Save DL models
for model_name, result in dl_results.items():
    filename = f"../results/models/dl/{model_name.lower()}.h5"
    result['model'].save(filename)
    print(f"Saved {model_name} to {filename}")

print("\n✅ All models saved successfully!")

## 📝 Summary

### Trained Models:
- ✅ 4 Machine Learning models
- ✅ 4 Deep Learning models

### Next Steps:
1. Proceed to **04_results_visualization.ipynb** for comprehensive evaluation
2. Analyze confusion matrices and ROC curves
3. Compare all models
4. Generate final report